# Chapter 6 - Collaborative Filtering

### The Framework

In [3]:
import pandas as pd
import numpy as np

In [5]:
#Load the u.user file into a dataframe
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv('data/u.user', sep='|', names=u_cols, encoding='latin-1')

users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


<div style="text-align:center;">
    <img src='images/pearson.jpg' width='500'>
</div>

### Singular Value Decomposition

Singular value decomposition, or SVD, is a type of matrix analysis technique that allows us to represent a high-dimensional matrix in a lower dimension.
SVD achieves this by identifying and removing the less important parts of the matrix and producing an approximation in the desired number of dimensions.

The SVD approach to collaborative filtering was first proposed by Simon Funk and proved to be extremely popular and effective during the Netflix
prize competition. We will use a black box implementation of the SVD collaborative filter as provided by the surprise package in the next chapter.